In [1]:
%pip install pycob

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pycob as cob

In [4]:
app = cob.App("")

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame([1,2,3])

In [9]:
app.to_cloud_pickle(df, 'test.pkl')

In [7]:
app.from_cloud_pickle('test.pkl')

,0
0,1
1,2
2,3


In [11]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [13]:
# Save output in a variable `df`

%%bigquery df --project pycob-prod
SELECT *
FROM `pycob-prod.samples.pypi_downloads_by_day`
WHERE date = '2023-01-31'
AND num_downloads_per_day > 100
ORDER BY num_downloads_per_day desc

Query is running:   0%|          |

Downloading:   0%|          |

In [15]:
# Upload a pickle of the dataframe to PyCob Cloud
app.to_cloud_pickle(df, 'top_projects.pkl')

In [17]:
# Save output in a variable `df`

%%bigquery pypi_projects_by_month --project pycob-prod
SELECT pypi_project, FORMAT_DATE("%Y-%m", date) as month, sum(num_downloads_per_day) / count(distinct date) as avg_downloads_per_day
FROM `pycob-prod.samples.pypi_downloads_by_day`
WHERE pypi_project in
(
SELECT pypi_project
FROM `pycob-prod.samples.pypi_downloads_by_day`
WHERE date = '2023-01-31'
AND num_downloads_per_day > 100
)
GROUP BY pypi_project, month

Query is running:   0%|          |

Downloading:   0%|          |

In [19]:
# Upload a pickle of the dataframe to PyCob Cloud
app.to_cloud_pickle(pypi_projects_by_month, 'pypi_projects_by_month.pkl')

In [9]:
pypi_projects_by_month = app.from_cloud_pickle('pypi_projects_by_month.pkl')

In [11]:
pypi_projects_by_month.query("pypi_project in ('boto3', 'google-api-core')")

,pypi_project,month,avg_downloads_per_day
47749,boto3,2023-01,1.773817e+07
90090,google-api-core,2022-11,6.504241e+06
100655,google-api-core,2022-10,6.462777e+06
101226,boto3,2022-10,1.302938e+07
118667,boto3,2022-08,1.286819e+07
147762,google-api-core,2023-01,6.321126e+06
148716,google-api-core,2022-12,6.367696e+06
149647,boto3,2022-12,1.641261e+07
154792,google-api-core,2023-02,8.191439e+06
157294,boto3,2023-02,1.879078e+07


In [12]:
import plotly.express as px

In [13]:
fig = px.line(pypi_projects_by_month.query("pypi_project in ('boto3', 'google-api-core')").sort_values(["month", "pypi_project"]), x="month", y="avg_downloads_per_day", line_group="pypi_project", color="pypi_project")

In [14]:
fig.show()